# Building Features from Light Curve Transformations using TSFRESH

The module TSFRESH can take multiple series of the same "type" the type here being a series of light curves. 

By creating multiple transformations of the dataset, and extracting only the strongest features from each we should be able to get a strong set of features. 

The following common time series transformations will be applied to both the TESSfield_05h_01d dataset and its equivalent phase fold:
- Forward Derivative
- Log(Forward Derivative)
- Log(data) 
- x/(Std.Dev(column)) 


In [6]:
# import the modules 
import pandas as pd
import numpy as np
import tsfresh
from tsfresh import extract_relevant_features
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import FeatureExtractionSettings


# open the main LC file that we will apply the transformations to

data = np.load("TESSfield_05h_01d.npy")
data_raw = np.delete(data, (0), axis=0)
df_raw = pd.DataFrame(data_raw)

# open Chelsea's combined feature file
# remove the last row to make the dimensions match with the raw LC file
data_combined_features = pd.read_csv("TESSfield_05h_01d_combinedfeatures.csv", header=0, index_col = 0)
data_combined_features = data_combined_features.drop(data_combined_features.index[-1])

# drop the columns that aren't features
X = data_combined_features.drop(['Ids', 'CatalogY', 'ManuleY', 'CombinedY',
                                 'Catalog_Period', 'Depth', 'Catalog_Epoch', 'SNR'],
                                axis=1)

data_phase = np.load("TESSfield_05h_01d_phasefold.npy")
data_phase_df = pd.DataFrame(data_phase)

# get the target values 
# In order to get the index of Y to match with X 
# we need to add one to the values 
y = pd.Series()
y = data_combined_features['CombinedY']
y.index = y.index + 1

/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


The first transformation will be the forward derivative transformation:

In [7]:
transpose = df_raw.T
transpose_phase = data_phase_df.T

derivative_df = pd.DataFrame()
derivative_df_phase = pd.DataFrame()

for i, columns in enumerate(transpose.columns):
    derivative_df[columns] = transpose[columns] - transpose[columns].shift(1)

for i, columns in enumerate(transpose_phase.columns):
        derivative_df_phase[columns] = transpose_phase[columns] - transpose_phase[columns].shift(1)


derivative_df = derivative_df.T
derivative_df_phase = derivative_df_phase.T

Next we'll do the logaritmic transformation, applying it to both the raw data and the derivative

In [8]:
log_raw_data = df_raw.apply(np.log)
log_difference = derivative_df.apply(np.log)

log_phase = data_phase_df.apply(np.log)
log_difference_phase = derivative_df_phase.apply(np.log)

The following is also another common transformation to apply; 
dividing the column values by the standard deviation of the column.

In [9]:
df_std_dev = (df_raw/df_raw.std(axis=0))
df_std_dev_phase = (data_phase_df/data_phase_df.std(axis=0))


# to confirm it worked uncomment below 

# print raw_data_frame[0][0]

# print 6.68758/raw_data_frame.std(axis=0)[0]

# print df_std_dev[0][0]

Now we'll apply the nessessary alterations to the data sets so that they can work with TSFRESH.  

In [11]:
Ids = data_combined_features['Ids']

df_raw['Ids'] = Ids
df_std_dev['Ids'] = Ids
log_raw_data['Ids'] = Ids
log_difference['Ids'] = Ids
derivative_df['Ids'] = Ids

data_phase_df['Ids'] = Ids
df_std_dev_phase['Ids'] = Ids
log_phase['Ids'] = Ids
log_difference_phase['Ids'] = Ids
derivative_df_phase['Ids'] = Ids



# the following initiates a column based time series 

def df_transformation(df_raw, df_phase=False):
    # the following initiates a column based time series 
    for i, j in enumerate(df_raw['Ids']):
        if i == 0:
            temp = [[j]]*(len(df_raw.columns) - 1)
        else:
            temp += [[j]]*(len(df_raw.columns) - 1)
    
    # we initiate the dataframe of the ID values we just created
    df_raw_transpose = pd.DataFrame(temp, columns=['Ids'])
    
    # the time series data now needs to be transposed to progress down the column
    # so we retrieve each value in a linear list instead of an array
    if df_phase == True:
        vals = []
        for rows in data_phase:
            for values in rows:
                vals.append(values)
        
        df_raw_transpose['x'] = vals
        time = range(0, 100)*len(data_phase)
        df_raw_transpose['time'] = time
        
        return df_raw_transpose
    else:
        vals = []
        for rows in data_raw:
            for values in rows:
                vals.append(values)
                
        df_raw_transpose['x'] = vals
        time = range(0, 480)*len(data_raw)
        df_raw_transpose['time'] = time
        
        return df_raw_transpose

In [14]:
# calling the transformation function on all the data

df_raw_T = df_transformation(df_raw)
df_std_dev_T = df_transformation(df_std_dev)
log_data_frame_T = df_transformation(log_raw_data)
log_difference_frame_T = df_transformation(log_difference)
derivative_df_T = df_transformation(derivative_df)

data_phase_df_T = df_transformation(data_phase_df, df_phase=True).dropna()
df_std_dev_phase_T = df_transformation(df_std_dev_phase, df_phase=True).dropna()
log_phase_T = df_transformation(log_phase, df_phase=True).dropna()
log_difference_phase_T = df_transformation(log_difference_phase, df_phase=True).dropna()
derivative_df_phase_T = df_transformation(derivative_df_phase, df_phase=True).dropna()

Note that feature extraction in the next step takes a handful of hours. 

In [15]:
# the following is the implementation of the tsfresh algorithm

from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute


total_dict = {'A': df_raw_T, 'B': df_std_dev_T, 'C': log_data_frame_T,
              'D': derivative_df_T, 'E': data_phase_df_T, 'F': df_std_dev_phase_T, 'G': log_phase_T, 
             'H': log_difference_phase_T, 'I': derivative_df_phase_T}

extracted_features = extract_features(total_dict, column_id="Ids",
                                      column_sort="time")


# impute removes any features that only contain NaN 
impute(extracted_features)

# features filtered removes features that aren't statistically significant 
# and puts them in format that allows them to be used for training
features_filtered = select_features(extracted_features, y)


/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/signal/spectral.py:772: UserWarning: nperseg = 256, is greater than input length = 71, using nperseg = 71
  'using nperseg = {1:d}'.format(nperseg, x.shape[-1]))
/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/signal/spectral.py:772: UserWarning: nperseg = 256, is greater than input length = 100, using nperseg = 100
  'using nperseg = {1:d}'.format(nperseg, x.shape[-1]))
/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/signal/spectral.py:772: UserWarning: nperseg = 256, is greater than input length = 95, using nperseg = 95
  'using nperseg = {1:d}'.format(nperseg, x.shape[-1]))
/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/signal

Note: The above error is to with SciPys handling of one of the features being calculated by TSFRESH. I don't understand the error entirely or its effects on the results. 

Below the Sklearn modules are called xgboost is tested on the default features generated by TSFRESH. 

In [18]:
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

# function to fit and get scores
def modelfit(alg, X, y, cv_folds=4):

    # StratifiedKFold automatically used by cross_val_predict on binary classification
    # does not use trapezoid rule
    # y_pred calculates the probabilities that each value is 1 or 0 using stratified cross validation 
    # pr_auc calculates the area under a precision recall curve 
    y_pred = cross_val_predict(alg, X, y, cv=cv_folds)
    pr_auc = metrics.average_precision_score(y, y_pred)


    # Print model report:
    print "pr_auc model score: {0}".format(pr_auc)

# initialize model and call fitting function
xgb1 = XGBClassifier(
    objective='binary:logistic')

modelfit(xgb1, features_filtered, y)

pr_auc model score: 0.65090799211


We can see that the transformations applied help, as the first attempt at tsfresh in previous notebooks gave a score around .60. 

Below, the 'fdr_level' is called where a lower value makes the filter more conservative and the default .05. 

In [19]:
from tsfresh.feature_selection import FeatureSignificanceTestsSettings
from tsfresh.feature_selection import select_features
settings = FeatureSignificanceTestsSettings()
settings.fdr_level = 0.01

features_filtered = select_features(extracted_features, y,
                                    feature_selection_settings=settings)


In [20]:
modelfit(xgb1, features_filtered, y)

pr_auc model score: 0.65090799211


So making it more conservative caused a slight decrease in score, what if we double the default score of .05? 

In [21]:
settings.fdr_level = 0.1
# select_features(X, y, feature_extraction_settings=settings)
features_filtered = select_features(extracted_features, y,
                                    feature_selection_settings=settings)

In [22]:
modelfit(xgb1, features_filtered, y)

pr_auc model score: 0.667510195876


We see that caused an increase, so we'll try a range from very conservative up to all the features and see what it yields. 

In [23]:
fdr_levels = [.001, .005, .01, .03, .05, .07, .09, .1, .3, .5, .7, .9, 1]
for fdr_level in fdr_levels:
    settings.fdr_level = fdr_level
    features_filtered = select_features(extracted_features, y,
                                        feature_selection_settings=settings)
    print 'Training using {0} features'.format(len(features_filtered.columns))
    modelfit(xgb1, features_filtered, y)


Training using 842 features
pr_auc model score: 0.654071987604
Training using 907 features
pr_auc model score: 0.647603053878
Training using 944 features
pr_auc model score: 0.65090799211
Training using 957 features
pr_auc model score: 0.646299150504
Training using 982 features
pr_auc model score: 0.657490880934
Training using 987 features
pr_auc model score: 0.657490880934
Training using 992 features
pr_auc model score: 0.657490880934
Training using 1001 features
pr_auc model score: 0.667510195876
Training using 1062 features
pr_auc model score: 0.670761989217
Training using 1076 features
pr_auc model score: 0.65735898207
Training using 1157 features
pr_auc model score: 0.641296793199
Training using 1172 features
pr_auc model score: 0.635982618976
Training using 1172 features
pr_auc model score: 0.635982618976


As we can see it performs better than our previous result, with a high score of ~.67 compared with the first TSFRESH notebook which had a score of .60 however requires many more features (1062). We'll see what happens when we use the combined feature file on the best outcome. 

In [25]:
X = data_combined_features.drop(['Ids', 'CatalogY', 'ManuleY', 'CombinedY',
                                 'Catalog_Period', 'Depth', 'Catalog_Epoch', 'SNR', 'BLS_Depth_1_0'],
                                axis=1)
settings.fdr_level = .3
features_filtered = select_features(extracted_features, y,
                                    feature_selection_settings=settings)
total_features = pd.concat([X, features_filtered], axis=1)

modelfit(xgb1, features_filtered, y)


pr_auc model score: 0.670761989217


Which we can see is lower than the TSRESH features alone. However the best score so far is ~.72 which is with the combined features without BLS_Depth